<a href="https://colab.research.google.com/github/barileao/SVM_Scania/blob/master/svm_scania.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalando packages

In [0]:
#!pip install pyod
#!pip install impyute
#pip install bayesian-optimization

# Imports

In [0]:
# importing all the necessary libraries
from google.colab import files
import io
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm 
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.impute import SimpleImputer,MissingIndicator
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix,f1_score,log_loss
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from datetime import datetime
#from bayes_opt import BayesianOptimization


# Acessar os dados no Drive


In [0]:
from google.colab import drive
drive.mount('/drive')

# Upload dos dados


In [0]:
filetrain = r'/drive/My Drive/IFES/TCC_Scania/Codes/aps_failure_training_set.csv'
filetest = r'/drive/My Drive/IFES/TCC_Scania/Codes/aps_failure_test_set.csv'

train = pd.read_csv(filetrain)
print('Number of data points in the train dataset', train.shape[0])
print('Number of features in the given train dataset', train.shape[1])
test = pd.read_csv(filetest)
print('Number of data points in the test dataset', test.shape[0])
print('Number of features in the given test dataset', test.shape[1])

In [0]:
# ----------------------Train--------------------------------

# Replacing all the class labels as follows:

# neg = 0
# pos = 1
class_labels_train = []
for x in tqdm(train.index):        # tqdm = barra de progresso
    if train['class'].loc[x] == 'neg':
        class_labels_train.append(0)
        
    else:
        class_labels_train.append(1)
        

train['class'] = class_labels_train

# Distrubution of class labels in train dataset
train['class'].value_counts()

# ----------------------Test--------------------------------
class_labels_test = []
for x in tqdm(test.index):        # tqdm = barra de progresso
    if test['class'].loc[x] == 'neg':
        class_labels_test.append(0)
        
    else:
        class_labels_test.append(1)
        

test['class'] = class_labels_test

# Distrubution of class labels in train dataset
test['class'].value_counts()




# Histograma

In [0]:
# Histogram plot of class labels of train dataset
plt.hist(train['class'])
plt.title("Histogram plot of class labels")
print("Percentage of negative class in the given dataset :", (train[train['class']== 0].shape[0] / train.shape[0]) * 100)
print("Percentage of positive class in the given dataset :", (train[train['class']== 1].shape[0] / train.shape[0]) * 100)


# Remover e substituir NAN e NA

In [0]:
# ----------------------Train--------------------------------
# Replacing the na and nan values with np.NaN
train.replace(to_replace='nan', value = np.NaN,inplace = True)

train.replace(to_replace = 'na', value = np.NaN, inplace = True)

columns_names = train.columns

train.describe(include='all')

# Dropping all the columns where the number of missing values are more than 42K (>70%)
#train.dropna(axis = 1, thresh=42000,inplace= True)
#train.head()

# ----------------------Test--------------------------------

test.replace(to_replace='nan', value = np.NaN,inplace = True)

test.replace(to_replace = 'na', value = np.NaN, inplace = True)

test = test[train.columns]
test.describe(include='all')


In [0]:
train.head(10)

# Retirar os Outliers para o treinamento

In [0]:
#coletando os outliers do train para 'trainol'
trainol = train.loc[(train['class'] ==1)]
#print(trainol)

testol = test.loc[(test['class'] ==1)]
#print(testol

#removendo os outliers para treinamento 
train= train[train['class'] != 1]

y_train = []
for i in range(59000):
  y_train.append(True)

train['class'].value_counts()
test= test[test['class'] != 1]

# Train data: mediana, média e mais frequente 

In [0]:
# using median
#impute_median = SimpleImputer(missing_values= np.nan, strategy='median',copy = True, verbose= 2)
#train_median = pd.DataFrame(impute_median.fit_transform(train),columns=train.columns)
#train_median['class'].value_counts()
#train_imputed_median.to_csv("Train_imputed_median")

#trainol_median = pd.DataFrame(impute_median.fit_transform(trainol),columns=trainol.columns)
#trainol_median['class'].value_counts()

# using mean
impute_mean = SimpleImputer(missing_values= np.nan, strategy='mean',copy = True, verbose= 2)
train_mean = pd.DataFrame(impute_mean.fit_transform(train),columns=train.columns)
train_mean['class'].value_counts()
#train_imputed_mean.to_csv("Train_imputed_mean")

trainol_mean = pd.DataFrame(impute_mean.fit_transform(trainol),columns=trainol.columns)
trainol_mean['class'].value_counts()

# using most frequent
#impute_most_frequent = SimpleImputer(missing_values= np.nan, strategy='most_frequent',copy = True, verbose= 2)
#train_most_frequent = pd.DataFrame(impute_most_frequent.fit_transform(train),columns=train.columns)
#train_imputed_most_frequent.to_csv("Train_imputed_most_frequent")

#trainol_most_frequent = pd.DataFrame(impute_most_frequent.fit_transform(trainol),columns=trainol.columns)
#trainol_most_frequent['class'].value_counts()

# Feature engineering, creating a new feature set which indicates 
# the missing value in a given row and column as True else False
#missing_impute = MissingIndicator()
#miss = missing_impute.fit_transform(train)
#train_miss_indi = pd.DataFrame(miss)
#train_miss_indi.to_csv('train_miss_indi.csv')

# ----------------------Test--------------------------------
# using median
#impute_median = SimpleImputer(missing_values= np.nan, strategy='median',copy = True, verbose= 2)
#test_median = pd.DataFrame(impute_median.fit_transform(test),columns=test.columns)
#test_median['class'].value_counts()
#train_imputed_median.to_csv("Train_imputed_median")

# using mean
#impute_mean = SimpleImputer(missing_values= np.nan, strategy='mean',copy = True, verbose= 2)
test_mean = pd.DataFrame(impute_mean.fit_transform(test),columns=test.columns)
#test_mean['class'].value_counts()
#testol_mean = pd.DataFrame(impute_mean.fit_transform(testol),columns=testol.columns)
#test_imputed_mean.to_csv("Test_imputed_mean")

# using most frequent
#impute_most_frequent = SimpleImputer(missing_values= np.nan, strategy='most_frequent',copy = True, verbose= 2)
#test_most_frequent = pd.DataFrame(impute_most_frequent.fit_transform(test),columns=test.columns)
#test_imputed_most_frequent.to_csv("Test_imputed_most_frequent")


# Padronizando os dados

In [0]:
# Get column names first
names = train_mean.columns

# Create the Scaler object
scaler = preprocessing.StandardScaler().fit(train_mean)

# Fit your data on the scaler object - train_imputed_median
#train_st = scaler.fit_transform(train_median)
#train_st = pd.DataFrame(train_st, columns=names)
#trainol_st = scaler.fit_transform(trainol_median)
#trainol_st = pd.DataFrame(trainol_st, columns=names)

# Fit your data on the scaler object - train_imputed_mean
train_st = scaler.fit_transform(train_mean)
train_st = pd.DataFrame(train_mean, columns=names)
trainol_st = scaler.fit_transform(trainol_mean)
trainol_st = pd.DataFrame(trainol_st, columns=names)

# Fit your data on the scaler object - train_imputed_most_frequent
#train_st = scaler.fit_transform(train_most_frequent)
#train_st = pd.DataFrame(train_most_frequent, columns=names)
#trainol_st = scaler.fit_transform(trainol_most_frequent)
#trainol_st = pd.DataFrame(trainol_st, columns=names)

# ----------------------Test--------------------------------

# Fit your data on the scaler object - train_imputed_median
#test_st = scaler.fit_transform(test_median)
#test_st = pd.DataFrame(test_st, columns=names)

test_st = scaler.fit_transform(test_mean)
test_st = pd.DataFrame(test_st, columns=names)
#testol_st = scaler.fit_transform(testol_mean)
#testol_st = pd.DataFrame(testol_st, columns=names)

#test_st = scaler.fit_transform(test_most_frequent)
#test_st = pd.DataFrame(test_st, columns=names)
#testol_st = scaler.fit_transform(testol_most_frequent)
#testol_st = pd.DataFrame(testol_st, columns=names)


# Retirar a coluna ['class'] do train e trainol

In [0]:
train_st = train_st.drop(['class'], axis =1)
trainol_st = trainol_st.drop(['class'], axis =1)
test_st = test_st.drop(['class'], axis =1)
#testol_st = testol_st.drop(['class'], axis =1)
#print(test_mean)

# PCA

In [0]:
#pca_variance = 0.95
pca = PCA(n_components=50)
pca.fit(train_st)
bestfeat_train_st = pca.transform(train_st)
bestfeat_train_st = pd.DataFrame(bestfeat_train_st)
#print(bestfeat_trainol_st)

bestfeat_trainol_st = pca.transform(trainol_st)
#print(bestfeat_trainol_st)

bestfeat_test_st = pca.transform(test_st)
#print(bestfeat_test_st)

#print('Number of components: ', pca.n_components_)

In [0]:
print(sum(pca.explained_variance_ratio_))
#print(pca.components_)

# Training: One-Class SVM

In [0]:
oneclass = svm.OneClassSVM()

#oneclass.fit(train_st)
#oneclass.fit(train_mean)
#oneclass.fit(train_most_frequent)

# SearchGrid

In [0]:
from sklearn.model_selection import GridSearchCV
#from sklearn import grid_search
params = [{'gamma': [0.01], 'nu': [0.16, 0.1], 'kernel': ['rbf']}]
newmodel = GridSearchCV(oneclass, params, scoring='precision', return_train_score=True, verbose=10)


In [49]:
newmodel.fit(train_st, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] gamma=0.01, kernel=rbf, nu=0.16 .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  gamma=0.01, kernel=rbf, nu=0.16, score=(train=1.000, test=0.000), total=38.9min
[CV] gamma=0.01, kernel=rbf, nu=0.16 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 45.8min remaining:    0.0s


[CV]  gamma=0.01, kernel=rbf, nu=0.16, score=(train=1.000, test=1.000), total=39.4min
[CV] gamma=0.01, kernel=rbf, nu=0.16 .................................


...

[CV]  gamma=0.01, kernel=rbf, nu=0.1, score=(train=1.000, test=0.000), total=40.8min
[CV] gamma=0.01, kernel=rbf, nu=0.1 ..................................


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 232.0min remaining:    0.0s


[CV]  gamma=0.01, kernel=rbf, nu=0.1, score=(train=1.000, test=0.000), total=41.0min


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 279.9min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 279.9min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=OneClassSVM(cache_size=200, coef0=0.0, degree=3,
                                   gamma='auto_deprecated', kernel='rbf',
                                   max_iter=-1, nu=0.5, random_state=None,
                                   shrinking=True, tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid=[{'gamma': [0.01], 'kernel': ['rbf'],
                          'nu': [0.16, 0.1]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='precision', verbose=10)

In [50]:
print(newmodel.best_estimator_)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.01, kernel='rbf',
            max_iter=-1, nu=0.16, random_state=None, shrinking=True, tol=0.001,
            verbose=False)


# Bayesian Optimization

In [0]:
#bounds = {'nu': (0.1, 1), 'gamma': (0.1, 1)}

#optimizer = BayesianOptimization(f=oneclass, pbounds=bounds, random_state=1)
#print(optimizer)
#optimizer.maximize(init_points=2, n_iter=5)
#print(optimizer.max)

In [0]:
#optimizer.maximize()
#print(optimizer.max)

# Salvar arquivo com Pickle

In [0]:
import pickle

# salvar arquivo de treino
filename = '/drive/My Drive/IFES/TCC_Scania/Redes treinadas/model_mean_sgcv1.sav'
pickle.dump(newmodel.best_estimator_, open(filename, 'wb'))

# carregar arquivo de treino
#newmodel = pickle.load(open('/drive/My Drive/IFES/TCC_Scania/Redes treinadas/modelpca_median_sgcv3.sav', 'rb'))
#result = loaded_model.score(X_test, Y_test)
#print(result)

# Predict

In [53]:
#teste = trainol_median

pred_train = newmodel.best_estimator_.predict(train_st)
pred_outliers = newmodel.best_estimator_.predict(trainol_st)
pred_test = newmodel.best_estimator_.predict(test_st)

#print('pred teste = ', pred_teste)

# tirado de: https://scikit-learn.org/stable/auto_examples/svm/plot_oneclass.html
n_error_train = pred_train[pred_train == -1].size
n_error_outliers = pred_outliers[pred_outliers == 1].size
n_error_test = pred_test[pred_test == -1].size
print('erro train = ', n_error_train)
print('erro outliers = ', n_error_outliers)
print('erro teste = ', n_error_test)


erro train =  8434
erro outliers =  0
erro teste =  15625
